In [ ]:
!pip install langchain_community
!pip install langchain
!pip install langchain_google_genai
!pip install python-dotenv
!pip install google.colab
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 20.9 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 52.8 MB/s eta 0:00:00


In [ ]:
from langchain_community.document_loaders import TextLoader


In [ ]:
txt_path = r"/content/XYZ_doc.txt"

loader = TextLoader(txt_path, encoding='utf-8')
docs = loader.load()
print(docs)

[Document(metadata={'source': '/content/XYZ_doc.txt'}, page_content='Currently, we offer loans exclusively to our loyal users who are California residents. Loans are issued by XYZ, Inc., which holds a California Finance Lenders License (CFL) and operates as a licensed lender in the state of California (NMLS ID #2156167). XYZ Terms and conditions apply. Approval is subject to credit eligibility and applicable regulations.\n\nIn other states, we do not provide consumer loans as defined by the Truth in Lending Act. Instead, our service grants early access to your earned wages through an Earned Wage Access (EWA) product. The amount of XYZ Advance you receive depends on your anticipated wages. There are no interest charges on a XYZ Advance, and we do not perform credit checks to determine eligibility. Additionally, we do not pull credit reports to determine eligibility for XYZ Advance.\n\nPlease note that XYZ Advance is not a consumer loan product, and to be eligible for XYZ Advance, you mu

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [ ]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=1000)
data= text_splitter.split_documents(docs)
print(len(data))

4


In [ ]:
for chunk in data:
  print(chunk.page_content)

Currently, we offer loans exclusively to our loyal users who are California residents. Loans are issued by XYZ, Inc., which holds a California Finance Lenders License (CFL) and operates as a licensed lender in the state of California (NMLS ID #2156167). XYZ Terms and conditions apply. Approval is subject to credit eligibility and applicable regulations.

In other states, we do not provide consumer loans as defined by the Truth in Lending Act. Instead, our service grants early access to your earned wages through an Earned Wage Access (EWA) product. The amount of XYZ Advance you receive depends on your anticipated wages. There are no interest charges on a XYZ Advance, and we do not perform credit checks to determine eligibility. Additionally, we do not pull credit reports to determine eligibility for XYZ Advance.

Please note that XYZ Advance is not a consumer loan product, and to be eligible for XYZ Advance, you must meet the following criteria:
In other states, we do not provide consum

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from dotenv import load_dotenv
from google.colab import userdata

In [ ]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=userdata.get('GEMINI_API_KEY'))

# Test embedding a query
vector = embeddings.embed_query("Eligibility criteria")
print(len(vector))
print(vector[0])

768
0.029401754960417747


In [ ]:
from langchain_community.vectorstores import FAISS
vectorstore = FAISS.from_documents(data, embeddings)
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})

query = "What are the loan offerings?"

results = retriever.get_relevant_documents(query)

for i, doc in enumerate(results, 1):
    print(f"Result {i}:\n{doc.page_content}\n{'-'*50}")

Result 1:
Currently, we offer loans exclusively to our loyal users who are California residents. Loans are issued by XYZ, Inc., which holds a California Finance Lenders License (CFL) and operates as a licensed lender in the state of California (NMLS ID #2156167). XYZ Terms and conditions apply. Approval is subject to credit eligibility and applicable regulations.

In other states, we do not provide consumer loans as defined by the Truth in Lending Act. Instead, our service grants early access to your earned wages through an Earned Wage Access (EWA) product. The amount of XYZ Advance you receive depends on your anticipated wages. There are no interest charges on a XYZ Advance, and we do not perform credit checks to determine eligibility. Additionally, we do not pull credit reports to determine eligibility for XYZ Advance.

Please note that XYZ Advance is not a consumer loan product, and to be eligible for XYZ Advance, you must meet the following criteria:
-------------------------------

/tmp/ipython-input-9-1905705837.py:7: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  results = retriever.get_relevant_documents(query)


In [ ]:
query = "Am I eligible for a loan?"

results = retriever.get_relevant_documents(query)

for i, doc in enumerate(results, 1):
    print(f"{i}.\t{doc.page_content}\n")

1.	Currently, we offer loans exclusively to our loyal users who are California residents. Loans are issued by XYZ, Inc., which holds a California Finance Lenders License (CFL) and operates as a licensed lender in the state of California (NMLS ID #2156167). XYZ Terms and conditions apply. Approval is subject to credit eligibility and applicable regulations.

In other states, we do not provide consumer loans as defined by the Truth in Lending Act. Instead, our service grants early access to your earned wages through an Earned Wage Access (EWA) product. The amount of XYZ Advance you receive depends on your anticipated wages. There are no interest charges on a XYZ Advance, and we do not perform credit checks to determine eligibility. Additionally, we do not pull credit reports to determine eligibility for XYZ Advance.

Please note that XYZ Advance is not a consumer loan product, and to be eligible for XYZ Advance, you must meet the following criteria:

2.	Please note that XYZ Advance is no

In [ ]:
query = "What is the eligibility criteria?"

results = retriever.get_relevant_documents(query)

for i, doc in enumerate(results, 1):
    print(f"{i}.\t{doc.page_content}\n")

1.	Please note that XYZ Advance is not a consumer loan product, and to be eligible for XYZ Advance, you must meet the following criteria:

1. State Eligibility
        Available in all states except:

           Nevada: XYZ Advance is currently unavailable due to state regulations. XYZ intends to obtain a license in the future. Other services (XYZ account, transfers, cashback) are still offered.

           South Carolina: XYZ Advance is suspended due to EWA regulations. XYZ intends to obtain a license in the future. Other services remain available.

           Connecticut: Advance is available but limited to $200.

2. Deposit Requirement
       You must receive at least $300 in payroll or benefits deposits into your XYZ account within the last 14 days.

3. Repayment History
       All previous advances must be fully repaid within 15 calendar days to remain eligible for new ones.

2.	Currently, we offer loans exclusively to our loyal users who are California residents. Loans are issued

STORING DIFFERENT USER DETAILS IN JSON FORMAT FOR USER SPECIFIC RESPONSES

In [ ]:
import json
import os
json_path= "/content/users.json"

user_info = {
    "user1": {
        "name": "Sruti Susarla",
        "location": "India",
        "email": "sruti.susarla@gmail.com"
    },

    "user2": {
        "name": "Marty Bird",
        "location": "Chicago",
        "email": "martybird@gmail.com"
    }

}

if not os.path.exists(json_path):
    with open(json_path, "w") as f:
        json.dump(user_info, f, indent=4)
    print("Created users.json")
else:
    print("users.json already exists")


Created users.json


In [ ]:
user_info={
    "user1": {
        "name": "Sruti Susarla",
        "location": "India",
        "email": "sruti.susarla@gmail.com"
    },

    "user2": {
        "name": "Marty Bird",
        "location": "Chicago",
        "email": "martybird@gmail.com"
    },
    "user3": {
        "name": "John Doe",
        "location": "California",
        "email": "john.doe@gmail.com"
    },
    "user4": {
        "name": "Riya Jain",
        "location": "South Carolina",
        "email": "jane.smith@gmail.com"
    }
}

with open(json_path, "w") as f:
  json.dump(user_info, f, indent=4)
print("Updated users.json")

Updated users.json


In [ ]:
with open(json_path, "r") as f:
        data = json.load(f)
for key, value in data.items():
        print(f"{key} → {value}")

user1 → {'name': 'Sruti Susarla', 'location': 'India', 'email': 'sruti.susarla@gmail.com'}
user2 → {'name': 'Marty Bird', 'location': 'Chicago', 'email': 'martybird@gmail.com'}
user3 → {'name': 'John Doe', 'location': 'California', 'email': 'john.doe@gmail.com'}
user4 → {'name': 'Riya Jain', 'location': 'South Carolina', 'email': 'jane.smith@gmail.com'}


In [ ]:
saved_questions= "/content/Questions.json"
K=5
def update(question):
  if not os.path.exists(saved_questions):
    with open(saved_questions,"w") as f:
      qdict= {}
  else:
    with open(saved_questions,"r") as f:
      qdict= json.load(f)
  qdict[question]=qdict.get(question,0)+1
  topquestions= sorted(qdict,key=qdict.get,reverse=True)[:K]
  top_in_dict={q:qdict[q] for q in topquestions}
  with open(saved_questions,"w") as f:
    json.dump(top_in_dict,f,indent=4)

def fetchtop():
  if not os.path.exists(saved_questions):
    return []
  with open(saved_questions,"r") as f:
    qdict=json.load(f)
  return list(qdict.keys())

GIVING SAMPLE USER QUERIES TO DISPLAY THE TOP K QUERIES

In [ ]:
query0= "What are the loan offerings?"
update(query0)
query1= "What are the loan offerings?"
update(query1)
query2= "What are the loan offerings?"
update(query2)
query3= "Am I eligible?"
update(query3)
query4= "Am I eligible?"
update(query4)
query5= "What is the eligibility criteria?"
update(query5)
query6= "What is the eligibility criteria?"
update(query6)
query7= "How much do I have to pay for the advance loan program ?"
update(query7)
top_5= fetchtop()
for q in top_5:
  print("-", q)

- What are the loan offerings?
- Am I eligible?
- What is the eligibility criteria?
- How much do I have to pay for the advance loan program ?


GIVING TOP K QUESTIONS + USER SPECIFIC QUERY TO GEMINI AND GETTING BACK THE GENERATED RESPONSE + FAQs FOR CONVERSATION CONTINUATION.

In [ ]:
!pip install google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 20.8 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.18
    Uninstalling google-ai-generativelanguage-0.6.18:
      Successfully uninstalled google-ai-generativelanguage-0.6.18
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-google-genai 2.1.5 requires google-ai-generativelanguage<0.7.0,>=0.6.18, but you have google-ai-generativelanguage 0.6.15 which is incompatible.


In [ ]:
import google.generativeai as genai
def gen_response(query,retriever,top_questions,GEMINI_API_KEY,user_id):
  user_info=get_user_details(user_id)
  name=user_info.get("name","Sruti Susarla")
  location=user_info.get("location","Unknown")
  email=user_info.get("email","not mentioned")

  context_docs=retriever.get_relevant_documents(query)
  context="\n\n".join([doc.page_content for doc in context_docs])
  top_k= "\n".join(f"->{q}" for q in top_questions)
  prompt= f"""
  You are an expert assisstant helping users understand text document content.
  Given,

  User Info:
  Name: {name}
  Location: {location}
  Email: {email}

  context:{context}
  user question:{query}
  Top 5 frequently asked:{top_questions}
  Systematically answer this using format-
  Then at the end of your response, add:
  ---
  What would you like next-
  {top_k} """

  genai.configure(api_key=GEMINI_API_KEY)
  model=genai.GenerativeModel("models/gemini-1.5-flash")
  response=model.generate_content(prompt)
  return response.text.strip()

In [ ]:
query="What are the loan offerings?"
top_5_questions=fetchtop()
update(query)
response=gen_response(query,retriever,top_5_questions,userdata.get('GEMINI_API_KEY'))
print(response)

XYZ offers two types of financial products:

1. **Loans:**  These are currently available only to loyal users who are California residents.  Loans are issued by XYZ, Inc., a licensed lender in California (NMLS ID #2156167).  Approval depends on credit eligibility and applicable regulations.  XYZ Terms and Conditions apply.

2. **XYZ Advance (EWA):** This is an earned wage access product offered in most states *except* Nevada (currently unavailable due to state regulations), South Carolina (currently suspended due to EWA regulations), and Connecticut (available but capped at $200).  It's not a loan as defined by the Truth in Lending Act; it provides early access to earned wages.  There are no interest charges, and credit checks are not performed.


What would you like next-
->Am I eligible?
->What is the eligibility criteria?
->How much do I have to pay for the advance loan program ?


In [ ]:
query="Am I eligible?"
top_5_questions=fetchtop()
update(query)
response=gen_response(query,retriever,top_5_questions,userdata.get('GEMINI_API_KEY'),user_id)
print(response)

TypeError: get_user_details() missing 1 required positional argument: 'json_path'

In [ ]:
user_id="user1"
import json
def get_user_details(user_id,json_path):
  with open(json_path,"r")as f:
    users=json.load(f)
  return users.get(user_id,{})

--------------------------------------------------------------------
  **25-06-2025**

In [ ]:
import os
import json

json_path = "/content/users_data.json"

user_info = {
    "user1": {
        "name": "Sruti Susarla",
                "state": "California",
                "prev_advance_paid": True,
                "salary_credited_recently": True
            },
            "user2": {
                "name": "Marty Bird",
                "location": "Texas",
                "prev_advance_paid": False,
                "salary_credited_recently": False
            },
            "user3": {
                "name": "Charlie Smith",
                "location": "California",
                "prev_advance_paid": True,
                "salary_credited_recently": False
            },
            "user4": {
                "name": "Jane Rose",
                "location": "Texas",
                "prev_advance_paid": True,
                "salary_credited_recently": True
            },
            "user5": {
                "name": "Ram Singh",
                "location": "California",
                "prev_advance_paid": False,
                "salary_credited_recently": True
            },
            "user6": {
                "name": "Bob Brown",
                "location": "California",
                "prev_advance_paid": False,
                "salary_credited_recently": False
            }
        }

# if not os.path.exists(json_path):
#     with open(json_path, "w") as f:
#         json.dump(user_info, f, indent=4)
#     print("Created users_data.json")
# else:
#     print("users_data.json already exists")

users_data.json already exists


In [ ]:
from langchain_community.vectorstores import FAISS
vectorstore = FAISS.from_documents(data, embeddings)
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})

NameError: name 'data' is not defined

In [ ]:
import google.generativeai as genai
def gen_response(query,retriever,json_path,GEMINI_API_KEY, user_details: dict=None):
  with open(json_path,"r") as f:
    user_data=json.load(f)

  context_docs = retriever.get_relevant_documents(query)
  top_chunks = "\n\n".join([doc.page_content for doc in context_docs])

  user_context = f"[USER DETAILS] {user_details}" if user_details else ""
  context = f"""
  {user_context}
  [TOP K MATCHES]
  {top_chunks}
"""

  prompt=f"""
You are an expert banking assisstant.
Use the given context to answer the user's question.
Context: {context}
User's question: "{query}"
If required ask for further information from user.
"""
  genai.configure(api_key=GEMINI_API_KEY)
  model=genai.GenerativeModel("models/gemini-1.5-flash")
  response=model.generate_content(prompt)
  return response.text.strip()

TEST CASES:
USER 1:
California + Paid + Salary received

In [ ]:
query="Am I eligible for a loan?"
user_id="user1"
json_path="/content/users_data.json"
gemini_api_key=userdata.get("GEMINI_API_KEY")
response = gen_response(query,retriever,user_id,json_path,gemini_api_key)

print(response)

Based on the provided information, you are a California resident and have had a previous advance paid.  To determine your loan eligibility, I need more information:

1. **Deposit Requirement:**  Have you received at least $300 in payroll or benefits deposits into your XYZ account within the last 14 days?  Please answer yes or no.

2. **Repayment History:** Was your previous advance fully repaid within 15 calendar days? Please answer yes or no.


Once I have this information, I can give you a more accurate assessment of your eligibility for an XYZ Advance (which is not a traditional loan but early access to wages).  Remember that even with a "yes" to both questions, final approval is subject to XYZ's credit eligibility and applicable regulations.


USER 2: no location eligibility + Not paid

In [ ]:
user_info = {
    "user1": {
        "name": "Sruti Susarla",
                "address": "California",
                "previous_advances_paid": True,
                "payroll_recieved_within_last_14_days": 200
            },
            "user2": {
                "name": "Marty Bird",
                "location": "Texas",
                "prev_advance_paid": False,
                "salary_credited_recently": False
            },
            "user3": {
                "name": "Charlie Smith",
                "location": "California",
                "prev_advance_paid": True,
                "salary_credited_recently": False
            },
            "user4": {
                "name": "Jane Rose",
                "location": "Texas",
                "prev_advance_paid": True,
                "salary_credited_recently": True
            },
            "user5": {
                "name": "Ram Singh",
                "location": "California",
                "prev_advance_paid": False,
                "salary_credited_recently": True
            },
            "user6": {
                "name": "Bob Brown",
                "location": "California",
                "prev_advance_paid": False,
                "salary_credited_recently": False
            }
        }

In [ ]:
query="Am I eligible for a XYZ advance?"
user_details=user_info["user1"]
gemini_api_key=userdata.get("GEMINI_API_KEY")
response = gen_response(query,retriever,json_path,gemini_api_key, user_details=user_details)

print(response)

Based on the provided information, it's unclear whether you're eligible for an XYZ Advance.  While you're in California, which is eligible, and your previous advances were paid, you only received $200 in payroll within the last 14 days.  The eligibility requirements state you need at least $300 in deposits.

To determine your eligibility, I need to know the total amount of deposits (payroll or benefits) you received in your XYZ account within the last 14 days.  Please provide this information.


USER 3: Paid + California + No salary credited

In [ ]:
query="Waht is the loan eligibilty criteria?"
gemini_api_key=userdata.get("GEMINI_API_KEY")
response = gen_response(query,retriever,json_path,gemini_api_key)

print(response)

Our loan eligibility criteria are as follows:  Currently, we only offer loans to our loyal users who are California residents.  Approval is subject to a credit check and is contingent upon meeting applicable regulations.  For loan terms and conditions, please refer to the XYZ Terms and Conditions.

If you reside outside of California, we offer an Earned Wage Access (EWA) product called XYZ Advance instead of a traditional loan.  Eligibility for XYZ Advance requires:

1. **State Eligibility:**  You must reside in a state where XYZ Advance is available.  Currently, it is unavailable in Nevada and suspended in South Carolina due to regulatory issues. In Connecticut, the maximum advance is limited to $200.

2. **Deposit Requirement:** You must have received at least $300 in payroll or benefit deposits into your XYZ account within the last 14 days.

3. **Repayment History:** All previous XYZ Advances must be fully repaid within 15 calendar days.

Please note that eligibility criteria are su

In [ ]:
query="Waht are the loan offerings?"
gemini_api_key=userdata.get("GEMINI_API_KEY")
response = gen_response(query,retriever,json_path,gemini_api_key)

print(response)

We offer two types of financial products:

1. **Loans:**  Currently, loans are available only to California residents who are loyal users.  These loans are issued by XYZ, Inc., a California licensed lender (NMLS ID #2156167). Approval is subject to a credit check and meeting our eligibility requirements.  XYZ Terms and Conditions apply.

2. **XYZ Advance (Earned Wage Access - EWA):** This product is available in most states, but *not* in Nevada (due to state regulations) or South Carolina (due to EWA regulations).  In Connecticut, the maximum advance is $200.  It's not a loan; it's early access to your earned wages.  There are no interest charges, and a credit check is not performed. Eligibility requires at least $300 in payroll or benefits deposited into your XYZ account within the past 14 days, and full repayment of all previous advances within 15 calendar days.

To determine your eligibility for either product, please refer to the XYZ Terms of Service or contact XYZ customer support

In [ ]:
query="Waht are the XYZ terms and conditions for loans?"
gemini_api_key=userdata.get("GEMINI_API_KEY")
response = gen_response(query,retriever,json_path,gemini_api_key)

print(response)

I cannot provide the specific terms and conditions for XYZ loans.  The provided text only states that XYZ, Inc. holds a California Finance Lenders License and operates as a licensed lender in California (NMLS ID #2156167), and that their terms and conditions apply.  To obtain the complete terms and conditions, you must refer to the XYZ Terms of Service or contact XYZ customer support.


In [ ]:
query="Am I eligible for a loan?"
user_id="user3"
json_path="/content/users_data.json"
gemini_api_key=userdata.get("GEMINI_API_KEY")
response = gen_response(query,retriever,user_id,json_path,gemini_api_key)

print(response)

Based on the provided information, I cannot definitively answer whether you are eligible for a loan.  The information states that loans are only offered to California residents, and you are a California resident. However,  the eligibility for a loan is also subject to a credit check, and this information is missing from your profile.  Additionally,  the provided data mentions "XYZ Advance," which is not a loan but an early access to wages.  Since you have a previous advance paid, this raises the question of whether you mean a traditional loan or another advance.

To determine your loan eligibility, I need the following information:

1. **Your credit score:**  This is crucial for loan eligibility.
2. **Clarification on your loan request:** Are you applying for a traditional loan or another XYZ Advance?

Once I have this information, I can provide a more accurate assessment of your eligibility.


USER 4: No location eligibility + Paid

In [ ]:
query="Am I eligible for a loan?"
user_id="user4"
json_path="/content/users_data.json"
gemini_api_key=userdata.get("GEMINI_API_KEY")
response = gen_response(query,retriever,user_id,json_path,gemini_api_key)

print(response)

Based on the provided information, you are not eligible for a loan from XYZ, Inc.  XYZ only offers loans to California residents.  You live in Texas.

However, you may be eligible for an XYZ Advance, which is not a loan but early access to your earned wages.  To determine your eligibility for an XYZ Advance, I need some additional information:

1. **Deposits:** Have you received at least $300 in payroll or benefits deposits into your XYZ account within the last 14 days?
2. **Previous Advance Repayment:** Was your previous advance fully repaid within 15 calendar days?

Please provide this information so I can assess your eligibility for an XYZ Advance.


USER 5 : California + Not paid

In [ ]:
query="Am I eligible for a loan?"
user_id="user5"
json_path="/content/users_data.json"
gemini_api_key=userdata.get("GEMINI_API_KEY")
response = gen_response(query,retriever,user_id,json_path,gemini_api_key)

print(response)

Based on the provided information, you are a California resident and have had salary credited recently.  This makes you potentially eligible for a loan through XYZ, Inc. However, to determine your eligibility definitively, I need more information:

1. **XYZ Account Details:** Do you have an XYZ account?  This is necessary to determine loan eligibility.  If so, how much money has been deposited into your XYZ account within the last 14 days?  (You must have received at least $300).

2. **Previous Advance Repayment:**  Have you ever taken out an XYZ Advance before? If so, were all previous advances fully repaid within 15 calendar days?

Once I have this information, I can assess your eligibility for a loan.  Please note that even with a positive answer to these questions, final approval is subject to a credit check and XYZ's terms and conditions.


USER 6: California + Not paid + No salary credited

In [ ]:
query="Am I eligible for a loan?"
user_id="user6"
json_path="/content/users_data.json"
gemini_api_key=userdata.get("GEMINI_API_KEY")
response = gen_response(query,retriever,user_id,json_path,gemini_api_key)

print(response)

Based on the information provided, I cannot definitively answer whether you are eligible for a loan.  The information states that loans are only offered to loyal users in California. While you live in California,  it's unclear if you are a "loyal user" as that term is not defined.

Furthermore, the information focuses heavily on "XYZ Advance," an earned wage access product, not a traditional loan.  To determine your eligibility for an XYZ Advance, I need the following information:

1. **Are you a loyal user of XYZ products?** (Please clarify what constitutes a "loyal user" in your understanding).
2. **Have you received at least $300 in payroll or benefits deposits into your XYZ account within the last 14 days?**
3. **Have you fully repaid all previous XYZ Advances within 15 calendar days?**

Once I have this information, I can provide a more accurate assessment of your eligibility for an XYZ Advance.  If you are interested in a traditional loan and not an advance, please contact XYZ cu